In [107]:
# Import Langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Other modules and packages
import os
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv


In [108]:
load_dotenv()

True

In [109]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

### Defining the LLM

In [110]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm.invoke("Tell me a joke about cats")

AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-02dd4db1-b8f3-40f7-8d6b-75861bc5bccd-0', usage_metadata={'input_tokens': 13, 'output_tokens': 20, 'total_tokens': 33})

# Processing the documents

### Load the PDF

In [111]:
loader = PyPDFLoader("data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf")
pages = loader.load()
pages

Ignoring wrong pointing object 18 0 (offset 0)


[Document(metadata={'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page': 0}, page_content='APPLIED COGNITIVE PSYCHOLOGYAppl. Cognit. Psychol.20: 139–156 (2006)Published online 31 October 2005 in Wiley InterScience(www.interscience.wiley.com) DOI: 10.1002/acp.1178Consequences of Erudite Vernacular Utilized Irrespectiveof Necessity: Problems with Using Long Words NeedlesslyDANIEL M. OPPENHEIMER*Princeton University, USASUMMARYMost texts on writing style encourage authors to avoid overly-complex words. However, a majorityof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to givethe impression of intelligence. This paper explores the extent to which this strategy is effective.Experiments 1–3 manipulate complexity of texts and ﬁnd a negative relationship between complex-ity and judged intelligence. This relationship held regardless of the quality of the original essay, andirrespective of the participants’ prior expectations of 

### Split the documents

In [112]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)
chunks

[Document(metadata={'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page': 0}, page_content='APPLIED COGNITIVE PSYCHOLOGYAppl. Cognit. Psychol.20: 139–156 (2006)Published online 31 October 2005 in Wiley InterScience(www.interscience.wiley.com) DOI: 10.1002/acp.1178Consequences of Erudite Vernacular Utilized Irrespectiveof Necessity: Problems with Using Long Words NeedlesslyDANIEL M. OPPENHEIMER*Princeton University, USASUMMARYMost texts on writing style encourage authors to avoid overly-complex words. However, a majorityof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to givethe impression of intelligence. This paper explores the extent to which this strategy is effective.Experiments 1–3 manipulate complexity of texts and ﬁnd a negative relationship between complex-ity and judged intelligence. This relationship held regardless of the quality of the original essay, andirrespective of the participants’ prior expectations of 

## Create Embeddings

In [113]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()

In [114]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance",
                            embeddings=embedding_function)


## Creating the database

In [115]:
import uuid
def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []

    unique_chunks = []
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks,
                                        ids=list(unique_ids),
                                        embedding=embedding_function,
                                        persist_directory = vectorstore_path)

    vectorstore.persist()

    return vectorstore

In [116]:
# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks,
                                 embedding_function=embedding_function,
                                 vectorstore_path="vectorstore_test")

/tmp/ipykernel_64540/1624783470.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## Query for relevant data

In [117]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

In [118]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the title of the paper?")
relevant_chunks

[]

In [119]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

## Generate responses

In [120]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text,
                                question="What is the title of the paper?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.



---

Answer the question based on the above context: What is the title of the paper?



In [121]:
llm.invoke(prompt)

AIMessage(content="I don't know.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 71, 'total_tokens': 75, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-7c5a3a98-cd98-4eaa-9392-844140504716-0', usage_metadata={'input_tokens': 71, 'output_tokens': 4, 'total_tokens': 75})

## Using Langchain Expression language

In [122]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

AIMessage(content="I don't know.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 68, 'total_tokens': 72, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_74ba47b4ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fe15356-93bc-438e-a839-5455d5309a2c-0', usage_metadata={'input_tokens': 68, 'output_tokens': 4, 'total_tokens': 72})

## Generate structured responses

In [123]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")

class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [124]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo, strict=True)
        )

rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")

ExtractedInfo(paper_title=AnswerWithSources(answer='Title of the Research Paper', sources="The title is explicitly mentioned in the context as 'Title of the Research Paper'.", reasoning='The title is directly stated in the provided context.'), paper_summary=AnswerWithSources(answer='This paper discusses the impact of X on Y and presents findings that suggest significant correlations between the two.', sources='The summary of the paper is described in the context as exploring the relationship between X and Y.', reasoning='The summary is derived from the main findings and focus outlined in the context.'), publication_year=AnswerWithSources(answer='2023', sources='The publication year is stated as 2023 in the context.', reasoning='The year of publication is explicitly mentioned in the retrieved context.'), paper_authors=AnswerWithSources(answer='John Doe, Jane Smith', sources='The authors are listed as John Doe and Jane Smith in the context.', reasoning='The authorship of the paper is cle

## Transform response into dataframe


In [125]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response.dict()])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

,paper_title,paper_summary,publication_year,paper_authors
answer,Title of the research paper,Summary of the research paper,Publication year of the research paper,Authors of the research paper
source,The title of the research paper is mentioned a...,The summary of the research paper is described...,The publication year is indicated as...,The authors of the research paper are listed a...
reasoning,The title is explicitly stated in the context ...,The summary is provided in the context and out...,The publication year is stated in the context.,The authors are mentioned in the context.
